In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

KeyboardInterrupt: ignored

In [ ]:
# Identify and specify the GPU as the device
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install pytorch_pretrained_bert
# !pip install transformers==3.5.1
import os
import logging
import argparse
import random
import json

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler



# from pytorch_pretrained_bert.tokenization import BertTokenizer
from transformers import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel, BertPreTrainedModel
from pytorch_pretrained_bert.optimization import BertAdam


logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# !pip show transformers

Name: transformers
Version: 3.5.1
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.6/dist-packages
Requires: tokenizers, requests, tqdm, dataclasses, regex, filelock, sacremoses, sentencepiece, protobuf, packaging, numpy
Required-by: 


# **Functions and classes**

In [ ]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_examples(lines, set_type):
    examples = []
    for (i, ids) in enumerate(lines["data"]):
        guid = "%s-%s" % (set_type, ids )
        text_a = lines["data"][ids]['tokens']
        label = lines["data"][ids]['labels']
        examples.append(
            InputExample(guid=guid, text_a=text_a, label=label) )
    return examples   

def read_json(input_file):
    """Reads a json file for tasks in sentiment analysis."""
    with open(input_file) as f:
        return json.load(f)
        
def get_train_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "train")

def get_dev_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "dev")    

def get_test_examples(data_dir):
    """See base class."""
    return create_examples(
        read_json(data_dir), "test")    
    
class BertForSequenceLabeling(BertPreTrainedModel):
    def __init__(self, config, num_labels=3):
        super(BertForSequenceLabeling, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, mode):
    """Loads a data file into a list of `InputBatch`s.""" #check later if we can merge this function with the SQuAD preprocessing 
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        if mode!="ae":
            tokens_a = tokenizer.tokenize(example.text_a)
        else: #only do subword tokenization.
            tokens_a, labels_a, example.idx_map= tokenizer.subword_tokenize([token.lower() for token in example.text_a], example.label )

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if mode!="ae":
            label_id = label_map[example.label]
        else:
            label_id = [-1] * len(input_ids) #-1 is the index to ignore
            #truncate the label length if it exceeds the limit.
            lb=[label_map[label] for label in labels_a]
            if len(lb) > max_seq_length - 2:
                lb = lb[0:(max_seq_length - 2)]
            label_id[1:len(lb)+1] = lb

        features.append(
                InputFeatures(
                        input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id))
    return features

class ABSATokenizer(BertTokenizer):     
    def subword_tokenize(self, tokens, labels): # for AE
        split_tokens, split_labels= [], []
        idx_map=[]
        for ix, token in enumerate(tokens):
            sub_tokens=self.wordpiece_tokenizer.tokenize(token)
            for jx, sub_token in enumerate(sub_tokens):
                split_tokens.append(sub_token)
                if labels[ix]=="B" and jx>0:
                    split_labels.append("I")
                else:
                    split_labels.append(labels[ix])
                idx_map.append(ix)
        return split_tokens, split_labels, idx_map    

# **Arguments**

In [ ]:
train_batch_size = 32            
num_train_epochs = 4      
max_seq_length = 100
learning_rate = 3e-5
warmup_proportion = 0.1
do_valid = True
pretrained_model = "bert-base-german-cased"

# **Final Preprocessing of train examples**


In [ ]:
label_list = ['O', 'B', 'I']

tokenizer = ABSATokenizer.from_pretrained(pretrained_model)
train_examples = get_train_examples(data_dir = "train.json")
num_train_steps = int(len(train_examples) / train_batch_size) * num_train_epochs

train_features = convert_examples_to_features(examples = train_examples, 
                                              label_list = label_list, 
                                              max_seq_length = max_seq_length, 
                                              tokenizer = tokenizer, 
                                              mode = "ae")

logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)


02/09/2021 13:15:27 - INFO - filelock -   Lock 140096976734136 acquired on /root/.cache/torch/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock


02/09/2021 13:15:28 - INFO - filelock -   Lock 140096976734136 released on /root/.cache/torch/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c.lock
02/09/2021 13:15:28 - INFO - __main__ -   ***** Running training *****
02/09/2021 13:15:28 - INFO - __main__ -     Num examples = 2
02/09/2021 13:15:28 - INFO - __main__ -     Batch size = 32
02/09/2021 13:15:28 - INFO - __main__ -     Num steps = 0


# **Validation**

In [ ]:
valid_examples = get_dev_examples(data_dir = "dev.json")
valid_features= convert_examples_to_features(
    valid_examples, label_list, max_seq_length, tokenizer, "ae")
valid_all_input_ids = torch.tensor([f.input_ids for f in valid_features], dtype=torch.long)
valid_all_segment_ids = torch.tensor([f.segment_ids for f in valid_features], dtype=torch.long)
valid_all_input_mask = torch.tensor([f.input_mask for f in valid_features], dtype=torch.long)
valid_all_label_ids = torch.tensor([f.label_id for f in valid_features], dtype=torch.long)
valid_data = TensorDataset(valid_all_input_ids, valid_all_segment_ids, valid_all_input_mask, valid_all_label_ids)

logger.info("***** Running validations *****")
logger.info("  Num orig examples = %d", len(valid_examples))
logger.info("  Num split examples = %d", len(valid_features))
logger.info("  Batch size = %d",  train_batch_size)

valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size= train_batch_size)    

best_valid_loss=float('inf')
valid_losses=[]

02/09/2021 13:15:34 - INFO - __main__ -   ***** Running validations *****
02/09/2021 13:15:34 - INFO - __main__ -     Num orig examples = 3
02/09/2021 13:15:34 - INFO - __main__ -     Num split examples = 3
02/09/2021 13:15:34 - INFO - __main__ -     Batch size = 32


# **Model Setup**

In [ ]:
model = BertForSequenceLabeling.from_pretrained("https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased.tar.gz", num_labels = len(label_list) )
model.cuda()
# Prepare optimizer
param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad==True]
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = 12 # num_train_steps
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr= learning_rate,
                     warmup= warmup_proportion,
                     t_total=t_total)

02/09/2021 13:15:37 - INFO - pytorch_pretrained_bert.file_utils -   https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased.tar.gz not found in cache, downloading to /tmp/tmpr6zyibdm
100%|██████████| 407595242/407595242 [00:13<00:00, 29927693.73B/s]
02/09/2021 13:15:51 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpr6zyibdm to cache at /root/.pytorch_pretrained_bert/4983b4079d9a070d00f914e5e669c1b1376f201367e3507366bc8a9de9d62fd0.d994c35add5d5ceb797a0bce40d17a4abb02c9c37b804e92c448d31d8de34734
02/09/2021 13:15:52 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/4983b4079d9a070d00f914e5e669c1b1376f201367e3507366bc8a9de9d62fd0.d994c35add5d5ceb797a0bce40d17a4abb02c9c37b804e92c448d31d8de34734
02/09/2021 13:15:52 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpr6zyibdm
02/09/2021 13:15:52 - INFO - pytorch_pretrained_bert.modeling -   loading archive file 

# **Actual training**

In [ ]:
global_step = 0
model.train()
for _ in range(num_train_epochs):
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        loss.backward()

        lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_this_step
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1
        #>>>> perform validation at the end of each epoch .
    if do_valid:
        model.eval()
        with torch.no_grad():
            losses=[]
            valid_size=0
            for step, batch in enumerate(valid_dataloader):
                batch = tuple(t.cuda() for t in batch) # multi-gpu does scattering it-self
                input_ids, segment_ids, input_mask, label_ids = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids)
                losses.append(loss.data.item()*input_ids.size(0) )
                valid_size+=input_ids.size(0)
            valid_loss=sum(losses)/valid_size
            logger.info("validation loss: %f", valid_loss)
            valid_losses.append(valid_loss)
        if valid_loss<best_valid_loss:
            torch.save(model, "model.pt" )
            best_valid_loss=valid_loss
        model.train()
if do_valid:
    with open("valid.json", "w") as fw:
        json.dump({"valid_losses": valid_losses}, fw)
else:
    torch.save(model, "model.pt")

/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
02/09/2021 13:16:27 - INFO - __main__ -   validation loss: 1.342445
02/09/2021 13:16:29 - INFO - __main__ -   validation loss: 0.399739
02/09/2021 13:16:30 - INFO - __main__ -   validation loss: 0.271355
02/09/2021 13:16:33 - INFO - __main__ -   validation loss: 0.325429


# **Test**

In [ ]:
eval_batch_size = 8
eval_examples = get_test_examples(data_dir = "test.json")
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer, "ae")

logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

model = torch.load("model.pt")
model.cuda()
model.eval()

full_logits=[]
full_label_ids=[]
for step, batch in enumerate(eval_dataloader):
    batch = tuple(t.cuda() for t in batch)
    input_ids, segment_ids, input_mask, label_ids = batch
    
    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.cpu().numpy()

    full_logits.extend(logits.tolist() )
    full_label_ids.extend(label_ids.tolist() )


with open("predictions_ae.json", "w") as fw:
    assert len(full_logits)==len(eval_examples)
    #sort by original order for evaluation
    recs={}
    for qx, ex in enumerate(eval_examples):
        recs[int(ex.guid.split("-")[1]) ]={"sentence": ex.text_a, "idx_map": ex.idx_map, "logit": full_logits[qx][1:]} #skip the [CLS] tag.
    full_logits=[recs[qx]["logit"] for qx in recs.keys()]
    raw_X=[recs[qx]["sentence"] for qx in recs.keys()]
    idx_map=[recs[qx]["idx_map"] for qx in recs.keys()]
    json.dump({"logits": full_logits, "raw_X": raw_X, "idx_map": idx_map}, fw)

02/09/2021 13:17:28 - INFO - __main__ -   ***** Running evaluation *****
02/09/2021 13:17:28 - INFO - __main__ -     Num examples = 2
02/09/2021 13:17:28 - INFO - __main__ -     Batch size = 8


In [ ]:
# lb=np.argmax(logit[1])
# lb
# logit[1]

[4.48275089263916, -1.4218887090682983, -1.606652855873108]